In [1]:
from collections import defaultdict
from GrabReleaseCommits import *
from StarHistory import *
from GetRepoFromDataset import *
from CodeCovReport import *
import numpy as np

In [2]:
github_repos = filter_github_repos('../data/github-ranking-2024-02-15.csv')
refinedRepo = list()
for repo in github_repos:
    username, repo_name, token, language = repo[0], repo[1], "7848dd6f-5308-43f6-a02f-e10e31118854", repo[2]
    repo_info = detect_coverage_tool_usage("github", username, repo_name, token, language)
    if repo_info != None:
        print(repo_info)
        refinedRepo.append(repo_info)

print(refinedRepo)
print(len(refinedRepo))

['github', 'facebook', 'react', False, True, 'JavaScript']
['github', 'vuejs', 'vue', True, True, 'TypeScript']
['github', 'trekhleb', 'javascript-algorithms', True, False, 'JavaScript']
['github', 'twbs', 'bootstrap', False, True, 'JavaScript']
['github', 'flutter', 'flutter', True, True, 'Dart']
['github', 'Significant-Gravitas', 'AutoGPT', True, False, 'JavaScript']
['github', 'airbnb', 'javascript', True, False, 'JavaScript']
['github', 'huggingface', 'transformers', True, False, 'Python']
['github', 'avelino', 'awesome-go', False, True, 'Go']
['github', 'facebook', 'react-native', False, True, 'C++']
['github', 'axios', 'axios', True, False, 'JavaScript']
['github', 'nodejs', 'node', True, False, 'JavaScript']
['github', 'mrdoob', 'three.js', False, True, 'JavaScript']
['github', 'mui', 'material-ui', True, False, 'TypeScript']
['github', 'ant-design', 'ant-design', True, True, 'TypeScript']
['github', 'iluwatar', 'java-design-patterns', False, True, 'Java']
['github', 'nvbn', 'th

['github', 'dwyl', 'learn-elixir', True, False, 'Elixir']
['github', 'elixir-wallaby', 'wallaby', True, False, 'Elixir']
['github', 'ueberauth', 'ueberauth', True, False, 'Elixir']
['github', 'akira', 'exq', False, True, 'Elixir']
['github', 'whitfin', 'cachex', False, True, 'Elixir']
['github', 'elixir-lang', 'ex_doc', False, True, 'Elixir']
['github', 'bencheeorg', 'benchee', False, True, 'Elixir']
['github', 'elixir-mint', 'mint', False, True, 'Elixir']
['github', 'ash-project', 'ash', False, True, 'Elixir']
['github', 'cabol', 'nebulex', False, True, 'Elixir']
['github', 'midarrlabs', 'midarr-server', True, False, 'Elixir']
['github', 'mirego', 'accent', False, True, 'Elixir']
['github', 'whatyouhide', 'redix', False, True, 'Elixir']
['github', 'mojotech', 'torch', False, True, 'Elixir']
['github', 'avelino', 'awesome-go', False, True, 'Go']
['github', 'fatedier', 'frp', False, True, 'Go']
['github', 'gin-gonic', 'gin', True, True, 'Go']
['github', 'moby', 'moby', True, False, 'Go'

['github', 'fastify', 'fastify', False, True, 'JavaScript']
['github', 'react-boilerplate', 'react-boilerplate', False, True, 'JavaScript']
['github', 'sequelize', 'sequelize', True, False, 'JavaScript']
['github', 'JuliaLang', 'julia', True, True, 'Julia']
['github', 'FluxML', 'Flux.jl', True, False, 'Julia']
['github', 'SciML', 'DifferentialEquations.jl', True, False, 'Julia']
['github', 'MakieOrg', 'Makie.jl', True, False, 'Julia']
['github', 'jump-dev', 'JuMP.jl', True, True, 'Julia']
['github', 'TuringLang', 'Turing.jl', True, True, 'Julia']
['github', 'GiovineItalia', 'Gadfly.jl', True, False, 'Julia']
['github', 'JuliaPlots', 'Plots.jl', True, False, 'Julia']
['github', 'alan-turing-institute', 'MLJ.jl', True, True, 'Julia']
['github', 'JuliaData', 'DataFrames.jl', True, True, 'Julia']
['github', 'FluxML', 'Zygote.jl', True, False, 'Julia']
['github', 'JuliaPy', 'PyCall.jl', True, False, 'Julia']
['github', 'JuliaPlots', 'UnicodePlots.jl', True, False, 'Julia']
['github', 'Julia

['github', 'python', 'cpython', True, False, 'Python']
['github', 'scikit-learn', 'scikit-learn', True, True, 'Python']
['github', 'localstack', 'localstack', False, True, 'Python']
['github', 'psf', 'requests', True, False, 'Python']
['github', 'scrapy', 'scrapy', True, True, 'Python']
['github', 'commaai', 'openpilot', True, False, 'Python']
['github', 'Textualize', 'rich', True, False, 'Python']
['github', 'pandas-dev', 'pandas', True, False, 'Python']
['github', 'getsentry', 'sentry', True, True, 'Python']
['github', 'psf', 'black', False, True, 'Python']
['github', 'geekan', 'MetaGPT', True, False, 'Python']
['github', 'mitmproxy', 'mitmproxy', True, True, 'Python']
['github', 'apache', 'airflow', True, True, 'Python']
['github', 'httpie', 'cli', True, False, 'Python']
['github', 'certbot', 'certbot', True, True, 'Python']
['github', 'python-poetry', 'poetry', True, False, 'Python']
['github', 'encode', 'django-rest-framework', True, False, 'Python']
['github', 'open-mmlab', 'mmde

['github', 'RocketChat', 'Rocket.Chat', True, True, 'TypeScript']
['github', 'prisma', 'prisma', True, False, 'TypeScript']
['github', 'ant-design', 'ant-design-pro', True, False, 'TypeScript']
['github', 'NervJS', 'taro', True, False, 'TypeScript']
['github', 'date-fns', 'date-fns', False, True, 'TypeScript']
['github', 'typeorm', 'typeorm', True, False, 'TypeScript']
['github', 'trpc', 'trpc', True, False, 'TypeScript']
['github', 'solidjs', 'solid', False, True, 'TypeScript']
['github', 'ReactiveX', 'rxjs', False, True, 'TypeScript']
['github', 'colinhacks', 'zod', True, False, 'TypeScript']
['github', 'ianstormtaylor', 'slate', True, False, 'TypeScript']
['github', 'mattermost', 'mattermost', True, False, 'TypeScript']
['github', 'cheeriojs', 'cheerio', False, True, 'TypeScript']
['github', 'calcom', 'cal.com', True, False, 'TypeScript']
['github', 'JedWatson', 'react-select', False, True, 'TypeScript']
['github', 'mobxjs', 'mobx', False, True, 'TypeScript']
['github', 'pnpm', 'pnp

In [2]:
Refined_DataSet = [['github', 'facebook', 'react', False, True, 'JavaScript'], ['github', 'vuejs', 'vue', True, True, 'TypeScript'], ['github', 'trekhleb', 'javascript-algorithms', True, None, 'JavaScript']]

In [3]:
# Get the commit sha's for each repository.

# release_tag_commits = []

# for repo in Refined_DataSet:
#     platform, username, repo_name, codecov_used, coverall_used = repo[0], repo[1], repo[2], repo[3], repo[4]
#     sha_list = retrieve_commit_hashes(username, repo_name)
    
#     if(len(sha_list) > 0):
#         release_tag_commits.append([platform, username, repo_name, codecov_used, coverall_used, sha_list])
    
#     for commit in sha_list:
#         sha_hash, commit_date = commit[0], commit[1]
#         release_tag_commits.append([sha_hash, commit_date])
        
# print(release_tag_commits)

In [4]:
# #Now refine from release_tag_commits to only have the latest 10 release commits for each repo.
# def refine_sha_from_repo(commit_sha_list, num_of_releases):
#     sha_list_length = len(commit_sha_list)
#     print(sha_list_length)
    
#     index_list = np.round(np.linspace(0, sha_list_length - 1, num_of_releases)).astype(int)
    
#     refined_sha_list = []
    
#     for index in index_list:
#         if(index > 0):
#             refined_sha_list.append(commit_sha_list[index])
    
#     return refined_sha_list
    
# num_of_releases = 10
# repo_with_ten_releases = []

# for repo in release_tag_commits:
#     platform, username, repo_name, codecov_used, coverall_used = repo[0], repo[1], repo[2], repo[3], repo[4]
#     commit_sha_list = repo[5]
#     refined_sha_list = refine_sha_from_repo(commit_sha_list, num_of_releases)
#     repo_with_ten_releases.append([platform, username, repo_name, codecov_used, coverall_used, refined_sha_list])
# print(repo_with_ten_releases)

In [ ]:
#[[platform, username, repo_name, codecov_used, coverall_used, [sha_list]], [...], ...]
codecov_API_token = "7848dd6f-5308-43f6-a02f-e10e31118854"
#7a7e792a6fd7668b37e475bf39e58fdb24e4ad81
#get_coverall_build(platform, username, repo_name, "787b343f674c72837209bdffd55c59682910d807")
#get_codecov_all_builds("github", "vuejs", "vue", codecov_API_token)

codecov_total_pages = get_codecov_total_pages("github", "vuejs", "vue", codecov_API_token)
ret_list = []
if(codecov_total_pages == 1):
    ret_list = get_codecov_all_builds("github", "vuejs", "vue", codecov_API_token, "TypeScript")
else:
    ret_list = get_codecov_all_builds("github", "storybookjs", "storybook", codecov_API_token, "TypeScript")
    print(ret_list)
    ret_list = get_coverall_all_builds("github", "storybookjs", "storybook", "TypeScript")
    print(ret_list)
    list = detect_coverage_tool_usage("github", "storybookjs", "storybook", codecov_API_token, "TypeScript")
    for i in list:
        print(i)
# for repo in repo_with_ten_releases:
#     platform, username, repo_name, codecov_used, coverall_used = repo[0], repo[1], repo[2], repo[3], repo[4]
#     commit_sha_list = repo[5]
    
#     for sha_value in commit_sha_list:
#         if(codecov_used):
#             get_codecov_build(platform, username, repo_name, codecov_API_token, sha_value)
#         if(coverall_used):
#             get_coverall_build(platform, username, repo_name, sha_value)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55


In [6]:
from datetime import datetime
date_list = []
username = ret_list[1][0]
repo_name = ret_list[1][1]
print(username)
print(repo_name)
for i in ret_list:
    original_date_string = i[4]
    # Parse the original date string
    original_date = datetime.strptime(original_date_string, '%Y-%m-%dT%H:%M:%SZ')

    # Format the date as 'year-month-date'
    formatted_date = original_date.strftime('%d-%m-%Y')
    date_list.append(formatted_date)
    print("Formatted date:", formatted_date)

vuejs
vue
Formatted date: 23-10-2015
Formatted date: 16-06-2015
Formatted date: 12-06-2015
Formatted date: 19-05-2015
Formatted date: 07-05-2015
Formatted date: 06-05-2015
Formatted date: 21-04-2015
Formatted date: 21-04-2015
Formatted date: 21-04-2015
Formatted date: 18-04-2015
Formatted date: 04-04-2015
Formatted date: 04-04-2015
Formatted date: 21-02-2015
Formatted date: 05-02-2015
Formatted date: 07-12-2014
Formatted date: 04-12-2014
Formatted date: 02-12-2014
Formatted date: 02-12-2014
Formatted date: 01-12-2014
Formatted date: 13-11-2014
Formatted date: 08-11-2014
Formatted date: 07-11-2014
Formatted date: 07-11-2014
Formatted date: 07-11-2014
Formatted date: 07-11-2014
Formatted date: 26-09-2014
Formatted date: 09-09-2014
Formatted date: 07-08-2014
Formatted date: 29-07-2014
Formatted date: 29-07-2014
Formatted date: 10-07-2014
Formatted date: 04-07-2014
Formatted date: 23-06-2014
Formatted date: 16-06-2014
Formatted date: 12-06-2014
Formatted date: 11-06-2014
Formatted date: 10

In [7]:
list_b = get_star_data(username, repo_name, date_list)
print(list_b)

[['23-10-2015', 6762], ['16-06-2015', 4381], ['12-06-2015', 4299], ['19-05-2015', 3848], ['07-05-2015', 3720], ['06-05-2015', 3708], ['21-04-2015', 3599], ['21-04-2015', None], ['21-04-2015', None], ['18-04-2015', 3517], ['04-04-2015', 3453], ['04-04-2015', None], ['21-02-2015', 3173], ['05-02-2015', 3045], ['07-12-2014', 2515], ['04-12-2014', 2490], ['02-12-2014', 2481], ['02-12-2014', None], ['01-12-2014', 2467], ['13-11-2014', 2295], ['08-11-2014', 2257], ['07-11-2014', 2252], ['07-11-2014', None], ['07-11-2014', None], ['07-11-2014', None], ['26-09-2014', 2018], ['09-09-2014', 1940], ['07-08-2014', 1714], ['29-07-2014', 1679], ['29-07-2014', None], ['10-07-2014', 1588], ['04-07-2014', 1566], ['23-06-2014', 1516], ['16-06-2014', 1501], ['12-06-2014', 1471], ['11-06-2014', 1468], ['10-06-2014', 1458], ['10-06-2014', None], ['10-06-2014', None], ['06-06-2014', 1441], ['03-06-2014', 1424], ['29-05-2014', 1399], ['29-05-2014', None], ['25-04-2014', 1184], ['26-03-2014', 1020], ['25-03-2

In [10]:
for i in range(len(ret_list)):
    ret_list[i].append(list_b[i])

print(ret_list)    

[['vuejs', 'vue', 94.06734366648851, '5f27148eea85a915b2f0bb5f1a131324d7133635', '2015-10-23T21:12:21Z', 'TypeScript', ['23-10-2015', 6762]], ['vuejs', 'vue', 99.9395222255821, 'e8d82d2840d314970568e7c576b100aed7d0cd07', '2015-06-16T17:02:26Z', 'TypeScript', ['16-06-2015', 4381]], ['vuejs', 'vue', 100.0, '098a3cdd3bfd5b059be879d03cdfceb6a640ffdf', '2015-06-12T19:06:23Z', 'TypeScript', ['12-06-2015', 4299]], ['vuejs', 'vue', 100.0, 'd257c81a5889d45012f6df39873fba3f8697f0cc', '2015-05-19T20:12:05Z', 'TypeScript', ['19-05-2015', 3848]], ['vuejs', 'vue', 100.0, 'd257c81a5889d45012f6df39873fba3f8697f0cc', '2015-05-07T19:22:49Z', 'TypeScript', ['07-05-2015', 3720]], ['vuejs', 'vue', 100.0, '7f7e36bd4a298623d656ca7cab5f23976756de15', '2015-05-06T21:16:55Z', 'TypeScript', ['06-05-2015', 3708]], ['vuejs', 'vue', 100.0, '6c841059d2893d383befeed0caf8090d5f0e8b88', '2015-04-21T20:53:39Z', 'TypeScript', ['21-04-2015', 3599]], ['vuejs', 'vue', 100.0, '563b0b965c4c6724729ddfd42598184f13c8b4f7', '2015

In [11]:
for i in ret_list:
    print(i)

['vuejs', 'vue', 94.06734366648851, '5f27148eea85a915b2f0bb5f1a131324d7133635', '2015-10-23T21:12:21Z', 'TypeScript', ['23-10-2015', 6762]]
['vuejs', 'vue', 99.9395222255821, 'e8d82d2840d314970568e7c576b100aed7d0cd07', '2015-06-16T17:02:26Z', 'TypeScript', ['16-06-2015', 4381]]
['vuejs', 'vue', 100.0, '098a3cdd3bfd5b059be879d03cdfceb6a640ffdf', '2015-06-12T19:06:23Z', 'TypeScript', ['12-06-2015', 4299]]
['vuejs', 'vue', 100.0, 'd257c81a5889d45012f6df39873fba3f8697f0cc', '2015-05-19T20:12:05Z', 'TypeScript', ['19-05-2015', 3848]]
['vuejs', 'vue', 100.0, 'd257c81a5889d45012f6df39873fba3f8697f0cc', '2015-05-07T19:22:49Z', 'TypeScript', ['07-05-2015', 3720]]
['vuejs', 'vue', 100.0, '7f7e36bd4a298623d656ca7cab5f23976756de15', '2015-05-06T21:16:55Z', 'TypeScript', ['06-05-2015', 3708]]
['vuejs', 'vue', 100.0, '6c841059d2893d383befeed0caf8090d5f0e8b88', '2015-04-21T20:53:39Z', 'TypeScript', ['21-04-2015', 3599]]
['vuejs', 'vue', 100.0, '563b0b965c4c6724729ddfd42598184f13c8b4f7', '2015-04-21T1

In [13]:
# Original data

# Check if the length of the list is less than or equal to 20
if len(ret_list) <= 20:
    result = ret_list
else:
    # Generate 10 uniformly spaced indices
    uniform_indices = np.linspace(0, len(ret_list) - 1, 10, dtype=int)
    # Retrieve the corresponding elements from the data
    result = [ret_list[i] for i in uniform_indices]

# Display the result
for i, item in enumerate(result, start=1):
    print(f"{i} {item}")

1 ['vuejs', 'vue', 94.06734366648851, '5f27148eea85a915b2f0bb5f1a131324d7133635', '2015-10-23T21:12:21Z', 'TypeScript', ['23-10-2015', 6762]]
2 ['vuejs', 'vue', 100.0, '6c3b290bb051e16ce5fc0897638b2eb4abae0214', '2015-04-18T07:15:54Z', 'TypeScript', ['18-04-2015', 3517]]
3 ['vuejs', 'vue', 100.0, '0df318c29d7e10bfa8e5c7ca31837af91ff3174c', '2014-12-01T00:26:35Z', 'TypeScript', ['01-12-2014', 2467]]
4 ['vuejs', 'vue', 94.16488222698072, 'd7576079f2236f057c69f5911c91c8eac7288789', '2014-08-07T03:30:12Z', 'TypeScript', ['07-08-2014', 1714]]
5 ['vuejs', 'vue', 94.225580140313, 'de8ec61312c996d6687d0f60aa308aec1eb665ad', '2014-06-10T16:24:21Z', 'TypeScript', ['10-06-2014', 1458]]
6 ['vuejs', 'vue', 94.2970073404856, '0faf0e6ac9754106a2b5c970ad635dcc3c3a8c5a', '2014-03-24T22:28:39Z', 'TypeScript', ['24-03-2014', 978]]
7 ['vuejs', 'vue', 95.12516469038208, 'a701b9f402f33bc14f7847253ac919664f6b1da8', '2014-02-20T23:00:31Z', 'TypeScript', ['20-02-2014', 725]]
8 ['vuejs', 'vue', 95.3125, '4fdeba